In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
source('tests.R')
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [3]:
read_csv("https://raw.githubusercontent.com/sgvjlee23/2023S1DSCI100group6/main/data/trial.csv") -> trial
slice(trial, 1:5)

Rows: 776 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): LOCATION_ID
dbl (17): Sector_score, PARA_A, SCORE_A, PARA_B, SCORE_B, TOTAL, numbers, Ma...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Sector_score,LOCATION_ID,PARA_A,SCORE_A,PARA_B,SCORE_B,TOTAL,numbers,Marks,Money_Value,MONEY_Marks,District,Loss,LOSS_SCORE,History,History_score,Score,Risk
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.89,23,4.18,6,2.50,2,6.68,5,2,3.38,2,2,0,2,0,2,2.4,1
3.89,6,0.00,2,4.83,2,4.83,5,2,0.94,2,2,0,2,0,2,2.0,0
3.89,6,0.51,2,0.23,2,0.74,5,2,0.00,2,2,0,2,0,2,2.0,0
3.89,6,0.00,2,10.80,6,10.80,6,6,11.75,6,2,0,2,0,2,4.4,1
3.89,6,0.00,2,0.08,2,0.08,5,2,0.00,2,2,0,2,0,2,2.0,0


In [4]:
trial_split <- initial_split(trial, prop = 0.75, strata = Risk)
trial_train <- training(trial_split)
trial_test <- testing(trial_split) 
glimpse(trial_train)

Rows: 581
Columns: 18
$ Sector_score  <dbl> 3.89, 3.89, 3.89, 3.89, 3.89, 3.89, 3.89, 3.89, 3.89, 3.…
$ LOCATION_ID   <chr> "6", "6", "6", "8", "3", "14", "5", "5", "5", "19", "19"…
$ PARA_A        <dbl> 0.00, 0.00, 0.00, 0.00, 0.94, 0.00, 0.00, 0.00, 0.00, 0.…
$ SCORE_A       <dbl> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,…
$ PARA_B        <dbl> 4.83, 0.08, 0.83, 0.84, 0.01, 1.03, 1.76, 2.97, 0.94, 0.…
$ SCORE_B       <dbl> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,…
$ TOTAL         <dbl> 4.83, 0.08, 0.83, 0.84, 0.95, 1.03, 1.76, 2.97, 0.94, 0.…
$ numbers       <dbl> 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,…
$ Marks         <dbl> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,…
$ Money_Value   <dbl> 0.940, 0.000, 2.950, 0.007, 0.100, 0.000, 1.080, 2.840, …
$ MONEY_Marks   <dbl> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,…
$ District      <dbl> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,…
$ Loss          <d

Introduction: This project focuses on predicting a firm's potential for fraud, leveraging both current and historical risk factors. In general, fraud involves carrying out deceptive actions such as manipulating and misrepresenting their financial activities and information, to secure an unfair or unlawful financial gain. To run an effective classification, we have decided to utilize the Trial dataset as our basis for classification. This dataset largely includes two set of risk factors, inherent risk(current) & control risk factors (historical). And based on these two sets, we want to determine whether present or historical factors will be more accurate in classifying the fraudulent risk of any given firm.

In [ ]:
fraud_count <- trial_train |>